# Problem description

In the telecom industry, customers are able to choose from multiple service providers and actively switch from one operator to another. In this highly competitive market, the telecommunications industry experiences an average of 15-25% annual churn rate. Given the fact that it costs 5-10 times more to acquire a new customer than to retain an existing one, customer retention has now become even more important than customer acquisition.



For many incumbent operators, retaining high profitable customers is the number one business goal. To reduce customer churn, telecom companies need to predict which customers are at high risk of churn.

## Definition of churn

There are two main models of payment in the telecom industry - postpaid (customers pay a monthly/annual bill after using the services) and prepaid (customers pay/recharge with a certain amount in advance and then use the services).

In the Indian and the southeast Asian market, approximately 80% of revenue comes from the top 20% customers (called high-value customers). Thus, if we can reduce churn of the high-value customers, we will be able to reduce significant revenue leakage.

## Goal of the analysis
 

Define high-value customers based on a certain metric (mentioned later below) and predict churn only on high-value customers.

### Import the analysis libraries

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


# matplotlib output formatting
%matplotlib inline

# pandas output formatting
pd.options.display.float_format = '{:.2f}'.format


# Utility fuction: line seperator
def print_ln():
    print('-'*80, '\n')



### Load the dataset

In [ ]:
# Load csv data file
telecom_orig = pd.read_csv('../data/raw/telecom_churn_data.csv', low_memory=False)
telecom_orig.head(5)

### Describe the dataset features

In [ ]:
telecom_orig.describe()

In [ ]:
# Understand the properties of loaded dataframe

telecom = telecom_orig.copy()
print('Dataframe Shape: ', telecom.shape); print_ln();
print("Dataframe Info: \n"); telecom.info(); print_ln();
telecom.head(5)

In [ ]:

def type_features(data):
    categorical_features = data.select_dtypes(include = ["object"]).columns
    numerical_features = data.select_dtypes(exclude = ["object"]).columns
    print( "categorical_features :",categorical_features)
    print_ln()
    print("numerical_features:",numerical_features)
    print_ln()
    return categorical_features, numerical_features

telecom_cat_features, telecom_num_features =  type_features(telecom)



## Missing value analysis and treatment
TODO
- Delete: Delete the missing values
- Imputing by a simple statistic: Replace the missing values by another value, commonly the mean, median, mode etc.
- Predictive techniques: Use statistical models such as k-NN, SVM etc. to predict and impute missing values

###  Missing Value Analysis (column wise)

- summing up the missing values (column-wise)

In [ ]:
telecom.isnull().sum().sort_values(ascending = False)

### Percentage of missing values (column-wise)

In [ ]:
round(100*(telecom.isnull().sum()/len(telecom.index)), 2)

### Columns with more than 70% missing values

In [ ]:

colmns_missing_data = round(100*(telecom.isnull().sum()/len(telecom.index)), 2)
colmns_missing_data[colmns_missing_data >= 70]

## Filtering high-value customers:

In the Indian and the southeast Asian market, approximately 80% of revenue comes from the top 20% customers (called high-value customers).

Thus, if we can reduce churn of the high-value customers, we will be able to reduce significant revenue leakage.

Below are the columns for that:

- total_rech_data_6
- total_rech_data_7
- av_rech_amt_data_6
- av_rech_amt_data_7

(All of the above columns has more than 70% missing values) 

In [ ]:
print(telecom.total_rech_data_6.describe()); print_ln()
print(telecom.total_rech_data_7.describe()); print_ln()
print(telecom.av_rech_amt_data_6.describe()); print_ln()
print(telecom.av_rech_amt_data_7.describe()); print_ln()

### As these columns could lead us to high-valued customer we can't drop them despite having more than 70% missing value. So, we have to look for Imputation and we have to look at the distribution to find out the value.

In [ ]:
# total_rech_data_6

plt.figure(1)
fig, ax = plt.subplots(figsize=(10,10))
plt.subplot(2,2,1)
plt.title("total_rech_data_6")
plt.hist(telecom.total_rech_data_6.dropna(),bins=25)

# total_rech_data_7

plt.subplot(2,2,2)
plt.title("total_rech_data_7")
plt.hist(telecom.total_rech_data_7.dropna(),bins=25)

# av_rech_amt_data_6

plt.subplot(2,2,3)
plt.title("av_rech_amt_data_6")
plt.hist(telecom.av_rech_amt_data_6.dropna(),bins=25)

# total_rech_data_7

plt.subplot(2,2,4)
plt.title("av_rech_amt_data_7")
plt.hist(telecom.av_rech_amt_data_7.dropna(),bins=25)

### Insights from the distribution graphs

- Variables are numeric.
- The minimum value is 1, and not zero,
- Missing ( indicating that no recharge was done in this month.)
- Thus, missing values can be imputed with 0.

In [ ]:
# imputing the total values
telecom.total_rech_data_6.fillna(value=0, inplace=True)
telecom.total_rech_data_7.fillna(value=0, inplace=True)
telecom.total_rech_data_8.fillna(value=0, inplace=True)
telecom.total_rech_data_9.fillna(value=0, inplace=True)

# imputing the average values
telecom.av_rech_amt_data_6.fillna(value=0, inplace=True)
telecom.av_rech_amt_data_7.fillna(value=0, inplace=True)
telecom.av_rech_amt_data_8.fillna(value=0, inplace=True)
telecom.av_rech_amt_data_9.fillna(value=0, inplace=True)

### Further on in the analysis, we can choose to impute missing values of these columns with 0.
- av_rech_amt_data
- arpu_2g
- arpu_3g
- count_rech_2g
- count_rech_3g
- max_rech_data
- total_rech_data
- fb_user
- night_pck_user



In [ ]:
# Since av_rech_amt_data_* features are important for getting the high-value customers,
#lets impute the missing av_rech_amt_data_* with 0

def imputeNan(data,imputeColList=False,missingColList=False):
    # Function impute the nan with 0
    # argument: colList, list of columns for which nan is to be replaced with 0

    if imputeColList:
        for col in [y + s for s in ['_6','_7','_8','_9'] for y in imputeColList]:
            data[col].fillna(0, inplace=True)
    else:
        for col in missingColList:
            data[col].fillna(0, inplace=True)

In [ ]:
imputeCol = ['av_rech_amt_data', 'arpu_2g', 'arpu_3g', 'count_rech_2g', 'count_rech_3g',
             'max_rech_data', 'total_rech_data','fb_user','night_pck_user']
imputeNan(telecom,imputeCol)


### Columns with more than 50% missing values

In [ ]:

colmns_missing_data = round(100*(telecom.isnull().sum()/len(telecom.index)), 2)
colmns_missing_data[colmns_missing_data >= 50]

In [ ]:
drop_data_columns = ['date_of_last_rech_data_6','date_of_last_rech_data_7','date_of_last_rech_data_8','date_of_last_rech_data_9']

telecom.drop(drop_data_columns, axis=1, inplace=True)

## Checking the shape of the dataset now

In [ ]:
telecom.shape

In [ ]:
def getMissingValues(missingCutoff):
    # Function to retun the columns with more than missingCutoff% missing values.
    # argument: missingCutoff, % values threshold for missing values
    missing = round(100*(telecom.isnull().sum()/telecom.shape[0]))
    print("There are {} features having more than {}% missing values".format(len(missing.loc[missing > missingCutoff]),missingCutoff))
    return missing.loc[missing > missingCutoff]


### Missing values per column expressed as % of total number of values > 5%

In [ ]:

getMissingValues(5)

### The featureset is missing the data for the month of september(9).

We can choose to impute these values with 0 as they are large data and dropping them would lead us to lack of data for analysis

In [ ]:
telecom.update(telecom[['onnet_mou_9', 'offnet_mou_9', 'roam_ic_mou_9', 'roam_og_mou_9', 'loc_og_t2t_mou_9', 'loc_og_t2m_mou_9', 'loc_og_t2f_mou_9', 'loc_og_t2c_mou_9', 'loc_og_mou_9', 'std_og_t2t_mou_9', 'std_og_t2m_mou_9', 'std_og_t2f_mou_9', 'std_og_t2c_mou_9', 'std_og_mou_9', 'isd_og_mou_9', 'spl_og_mou_9', 'og_others_9', 'loc_ic_t2t_mou_9', 'loc_ic_t2m_mou_9', 'loc_ic_t2f_mou_9', 'loc_ic_mou_9', 'std_ic_t2t_mou_9', 'std_ic_t2m_mou_9', 'std_ic_t2f_mou_9', 'std_ic_t2o_mou_9', 'std_ic_mou_9', 'spl_ic_mou_9', 'isd_ic_mou_9', 'ic_others_9']].fillna(0))

In [ ]:
telecom.shape

In [ ]:
# Missing values per column expressed as % of total number of values
getMissingValues(2)

### Let's drop these customers from the data.

In [ ]:
missingcol = list(getMissingValues(2).index)
telecom=telecom[~telecom[missingcol].isnull().all(axis=1)]
telecom.shape


### For other customers where these missing values are spread out, let's impute them with 0. 

In [ ]:


missingcol.remove('date_of_last_rech_8')
missingcol.remove('date_of_last_rech_9')

imputeNan(telecom,missingColList=missingcol)

In [ ]:
getMissingValues(0)


### Some of these features take only one value. Lets impute their missing values in these features with the `mode` of the distribution

In [ ]:

col = ['loc_og_t2o_mou','std_og_t2o_mou','loc_ic_t2o_mou','last_date_of_month_7','last_date_of_month_8','last_date_of_month_9']
for c in col:
    telecom[c].fillna(telecom[c].mode()[0], inplace=True)


### Missing values per column expressed as % of total number of values


In [ ]:
getMissingValues(0)

### All the 4 columns have dates of `june, july, aug, sep` respectively. So we can easily impute them.

In [ ]:
telecom[telecom['date_of_last_rech_6'].isnull()]['date_of_last_rech_6'] = '6/30/2014'
telecom[telecom['date_of_last_rech_7'].isnull()]['date_of_last_rech_7'] = '7/31/2014'
telecom[telecom['date_of_last_rech_8'].isnull()]['date_of_last_rech_8'] = '8/31/2014'
telecom[telecom['date_of_last_rech_9'].isnull()]['date_of_last_rech_9'] = '9/30/2014'

### There are some columns whose only value is 0. Lets look at them.

In [ ]:
zero_columns=telecom.columns[(telecom == 0).all()]
zero_columns


### We can remove these columns as they are of no use.

In [ ]:
telecom.drop(zero_columns,axis=1,inplace=True)

In [ ]:
# Percentage of data left after removing the missing values.

print("Percentage of data remaining after treating missing values: {}%".format(round(telecom.shape[0]/99999 *100,2)))
print ("Number of customers: {}".format(telecom.shape[0]))
print ("Number of features: {}".format(telecom.shape[1]))

##### Fixing the data-type and column names

In [ ]:
telecom.reset_index(inplace=True,drop=True)

# list of all columns which have date

date_columns = list(telecom.filter(regex='date').columns)
date_columns

In [ ]:
# Converting dtype of date columns to datetime

for col in date_columns:
    telecom[col] = pd.to_datetime(telecom[col], format='%m/%d/%Y')

#####  Some monthly features which are not in the standard naming (_6,_7,_8,_9) :

- 'jun_vbc_3g' : 'vbc_3g_6'
- 'jul_vbc_3g' : 'vbc_3g_7'
- 'aug_vbc_3g' : 'vbc_3g_8'
- 'sep_vbc_3g' : 'vbc_3g_9'

In [ ]:
telecom.rename(columns={'jun_vbc_3g' : 'vbc_3g_6', 'jul_vbc_3g' : 'vbc_3g_7', 'aug_vbc_3g' : 'vbc_3g_8',
                      'sep_vbc_3g' : 'vbc_3g_9'}, inplace=True)

#### Filter high-value customers
Defining high-value customers as follows:

Those who have recharged with an amount more than or equal to X, where X is the 70th percentile of the average recharge amount in the first two months (the good phase).

##### Creating New Features

vol_data_mb_6', 'vol_data_mb_7', 'vol_data_mb_8', 'vol_data_mb_9'

These will store the total data volume (= vol_2gmb + vol_3gmb) used by user.

In [ ]:
#Creating new feature: 'vol_data_mb_6', 'vol_data_mb_7', 'vol_data_mb_8', 'vol_data_mb_9',

for i in range(6,10):
    telecom['vol_data_mb_'+str(i)] = (telecom['vol_2g_mb_'+str(i)]+telecom['vol_3g_mb_'+str(i)]).astype(int)

### These will store the average recharge value for each customer for every month
avg_rech_amt_6,avg_rech_amt_7,avg_rech_amt_8,avg_rech_amt_9



In [ ]:
# Creating new feature: avg_rech_amt_6,avg_rech_amt_7,avg_rech_amt_8,avg_rech_amt_9
for i in range(6,10):
    telecom['avg_rech_amt_'+str(i)] = round(telecom['total_rech_amt_'+str(i)]/telecom['total_rech_num_'+str(i)]+1,2)

In [ ]:
imputeNan(telecom,missingColList=['avg_rech_amt_6','avg_rech_amt_7','avg_rech_amt_8','avg_rech_amt_9'])

### These will store the total number of data recharge (=count_rech_2g + count_rech_3g ) for each month.

total_rech_num_data_6,total_rech_num_data_7,total_rech_num_data_8,total_rech_num_data_9



In [ ]:
#Creating new feature: total_rech_num_data_6,total_rech_num_data_7,total_rech_num_data_8,total_rech_num_data_9

for i in range(6,10):
    telecom['total_rech_num_data_'+str(i)] = (telecom['count_rech_2g_'+str(i)]+telecom['count_rech_3g_'+str(i)]).astype(int)

### These will store the total amount of data recharge (=total_rech_num_data * av_rech_amt_data ) for each month.
total_rech_amt_data_6,total_rech_amt_data_7,total_rech_amt_data_8,total_rech_amt_data_9



In [ ]:
#Creating new feature: total_rech_amt_data_6,total_rech_amt_data_7,total_rech_amt_data_8,total_rech_amt_data_9

for i in range(6,10):
    telecom['total_rech_amt_data_'+str(i)] = telecom['total_rech_num_data_'+str(i)]*telecom['av_rech_amt_data_'+str(i)]

### These will store the total recharge amount (= total_rech_amt + total_rech_amt_data ) for each customer, for each month.

total_month_rech_6,total_month_rech_7,total_month_rech_8,total_month_rech_9



In [ ]:
#Creating new feature: total_mon_rech_6,total_mon_rech_7,total_mon_rech_8,total_mon_rech_9
for i in range(6,10):
    telecom['total_month_rech_'+str(i)] = telecom['total_rech_amt_'+str(i)]+telecom['total_rech_amt_data_'+str(i)]

In [ ]:
# calculating the average of first two months (good phase) total monthly recharge amount

avg_goodPhase =(telecom.total_month_rech_6 + telecom.total_month_rech_7)/2

# finding the cutoff which is the 70th percentile of the good phase average recharge amounts

hv_cutoff= np.percentile(avg_goodPhase,70)

# Filtering the users whose good phase avg. recharge amount >= to the cutoff of 70th percentile.

hv_users = telecom[avg_goodPhase >=  hv_cutoff]
hv_users.reset_index(inplace=True,drop=True)

print("Number of High-Value Customers in the Dataset: %d\n"% len(hv_users))
print("Percentage High-value users in data : {}%".format(round(len(hv_users)/telecom.shape[0]*100),2))


##### Tagging Churners
churned customers (churn=1, else 0) based on the fourth month as follows:

Those who have not made any calls (either incoming or outgoing) AND have not used mobile internet even once in the churn phase. The attributes we need to use to tag churners are:

- total_ic_mou_9
- total_og_mou_9
- vol_2g_mb_9
- vol_3g_mb_9

- Function to tag customers as churners (churn=1, else 0) based on 'vol_2g_mb_','vol_3g_mb_','total_ic_mou_','total_og_mou_'
- Argument: churnPhaseMonth, indicating the month number to be used to define churn (default= 9)

In [ ]:
def getChurnStatus(data,churnPhaseMonth=9):
    
    churn_features= ['vol_2g_mb_','vol_3g_mb_','total_ic_mou_','total_og_mou_']
    flag = ~data[[s + str(churnPhaseMonth) for s in churn_features ]].any(axis=1)
    flag = flag.map({True:1, False:0})
    return flag

In [ ]:
hv_users['churn'] = getChurnStatus(hv_users,9)
print("There are {} users tagged as churners out of {} High-Value Customers.".format(len(hv_users[hv_users.churn == 1]),hv_users.shape[0]))
print_ln()
print("High-value Churn Percentage : {}%".format(round(len(hv_users[hv_users.churn == 1])/hv_users.shape[0] *100,2)))

### There are just 8.09% churn cases.
This indicated an highly imbalanced data set where the churn cases are the minority(8.14%) as opposed to the non-churners who are the majority(91.91)

##### Identify columns that have no varience & Drop

In [ ]:
# Identify columns that have no varience
hv_users_unique_count = hv_users.nunique().sort_values(ascending=False)
hv_users_unique_count

In [ ]:
# Identify bad colums that has no information (all entries are NA or same)
# Find columns with all NULL entries and add to drop_columns list
hv_users_unique_count_is_zero = hv_users_unique_count[hv_users_unique_count == 0]
print("Dataframe Unique Value Count is ZERO (all null values): \n", hv_users_unique_count_is_zero); print_ln();

drop_no_varience_columns = list(hv_users_unique_count_is_zero.index)
drop_no_varience_columns

In [ ]:
# Find columns with all same entries and add to drop_columns list

hv_users_unique_count_is_one = hv_users_unique_count[hv_users_unique_count == 1]
print("Dataframe Unique Value Count is ONE (all same values): \n", hv_users_unique_count_is_one); print_ln();

drop_no_varience_columns += list(hv_users_unique_count_is_one.index)

In [ ]:
hv_users.drop(drop_no_varience_columns, axis=1, inplace=True)

print('Dataframe Shape: ', hv_users.shape); print_ln();
print('Number of columns dropped here  = ', len(drop_no_varience_columns))
print('Dataframe Shape: ', hv_users.shape); print_ln();

In [ ]:
hv_users.head()

##### ROW-WISE: MISSING VALUES

In [ ]:
# sum it up to check how many rows have all missing values
print("Rows with all NULL values =",  hv_users.isnull().all(axis=1).sum())

# sum of misisng values in each row
rows_missing_data = hv_users.isnull().sum(axis=1)
rows_missing_data[rows_missing_data > 0].max()

### Therefore there is no need to drop the values.

In [ ]:
# Missing Values Treatment: continue
# look at the COLUMN summary again
df = round(100*(hv_users.isnull().sum()/len(hv_users.index)), 2)
df[df > 0]

### Note: The remaining missing values are in date columns, before imputing convert them into appropreate values for analysis.

In [ ]:
# covert date columns to python datetime format
date_vars = ["date_of_last_rech_6",  "date_of_last_rech_7", "date_of_last_rech_8"]
hv_users[date_vars].head()

In [ ]:
# convert to datetime
for col in date_vars:
    hv_users[col] = pd.to_datetime(hv_users[col])

print(hv_users[date_vars].info())
hv_users[date_vars].head()

In [ ]:
# Create new days columns, instead of date
import datetime
last_date_of_month_6 = datetime.datetime.strptime("30-06-2014", "%d-%m-%Y")
last_date_of_month_7 = datetime.datetime.strptime("31-07-2014", "%d-%m-%Y")
last_date_of_month_8 = datetime.datetime.strptime("31-08-2014", "%d-%m-%Y")

hv_users["rech_days_left_6"]      = (last_date_of_month_6 - hv_users.date_of_last_rech_6).astype('timedelta64[D]')
hv_users["rech_days_left_7"]      = (last_date_of_month_7 - hv_users.date_of_last_rech_7).astype('timedelta64[D]')
hv_users["rech_days_left_8"]      = (last_date_of_month_8 - hv_users.date_of_last_rech_8).astype('timedelta64[D]')

day_columns = ["rech_days_left_6", "rech_days_left_7", "rech_days_left_8"]

print(hv_users[day_columns].info())

In [ ]:
# Drop all old date columns: add dates columns to drop_column list
hv_users.drop(date_vars, axis=1, inplace=True)
telecom1 = hv_users.copy()

print('Number of columns droped here  = ', len(date_vars))
print('Dataframe Shape: ', hv_users.shape)

In [ ]:
# Missing Values Treatment: continue
# look at the COLUMN summary again
df = round(100*(hv_users.isnull().sum()/len(hv_users.index)), 2)
df[df > 0]

In [ ]:
hv_users[['rech_days_left_6', 'rech_days_left_7', 'rech_days_left_8']].describe()


In [ ]:
# Mean & Median are different: imputing with median value for recharge days left 
hv_users.loc[np.isnan(hv_users['rech_days_left_6']), ['rech_days_left_6']] = hv_users['rech_days_left_6'].median()
hv_users.loc[np.isnan(hv_users['rech_days_left_7']), ['rech_days_left_7']] = hv_users['rech_days_left_7'].median()
hv_users.loc[np.isnan(hv_users['rech_days_left_8']), ['rech_days_left_8']] = hv_users['rech_days_left_8'].median()


In [ ]:

print("\nTotal Number of missing values in the data frame = {}".format(hv_users.isnull().sum().sum()))


### There are 802 missing values so we can drop them directly as it will not have much effect on the dataset

In [ ]:
hv_users=hv_users.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
hv_users.shape

### Outlier treatment

In [ ]:
# Checking outliers at 25%,50%,75%,90%,95% and 99%
hv_users.describe(percentiles=[.25,.5,.75,.90,.95,.99])

- Most of the outliers are on the revenue data and the model being built is on usage of voice and data.
- Hence there is no need to treat these outliers

##### Save the cleaned data in new file

In [ ]:
# write treated telecom file
hv_users.to_csv("../data/processed/01_EDA.csv", sep=',', index=False)

